##### Short bias correction method validation exercise. where we check that we properly bias corrected the CMIP6 model output. #####

##### Check (1) For each cell in a small selection we calculate a range of quantiles (across the temporal distribution of a given cell) of tasmax within the whole historical period and we compare that to the reference ERA-5 data. ######

##### Check (2) Similarly, we calculate quantiles for the non-bias-corrected and bias-corrected future model output, compute the absolute change relative to historical and verify the changes are preserved after correction. ######

#### Set up 

In [1]:
%%capture
! pip install xclim
import numpy as np
from urbanspoon import core
import json
import gcsfs
import xarray as xr

In [4]:
gcm = 'GFDL-ESM4'
data_paths_file = '/home/jovyan/output/tasmax_gcms_data_paths.json'
with open(data_paths_file) as json_file:
    data_dict = json.load(json_file)[gcm]

In [5]:
def read_gcs_zarr(zarr_url, token='/opt/gcsfuse_tokens/impactlab-data.json', check=False, consolidated=True):
    """
    takes in a GCSFS zarr url, bucket token, and returns a dataset 
    Note that you will need to have the proper bucket authentication. 
    """
    fs = gcsfs.GCSFileSystem(token=token)
    store_path = fs.get_mapper(zarr_url, check=check) 
    ds = xr.open_zarr(store_path, consolidated=consolidated) 
    ds.close() 
    return ds 

#### Select a few grid cells and quantiles to look at ###

In [6]:
# Andorra, Ohio, N Brazil
cells = [(41.5, 1.5),(39.5, -83.5),(-5.5, -49.5)]
myquants = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
quants_results = {}

#### Compute quantiles for each dataset ###

In [11]:
data_files = [
    data_dict['coarse']['cmip6']['historical'],
    data_dict['coarse']['cmip6']['ssp370'],
    data_dict['coarse']['bias_corrected']['historical'],
    data_dict['coarse']['bias_corrected']['ssp370'],
    data_dict['coarse']['ERA-5']
]
for data_file in data_files:
    da = read_gcs_zarr(data_file)['tasmax'].chunk(dict(time=-1))
    if data_file == data_files[1] or data_file == data_files[3]:
        da = da.sel(time=slice('2080', '2100'))
    result = core.xr_quantiles_across_time_by_cell(da=da, q=myquants, cells=cells)
    for r,k in result.items():
        result[r] = k.compute()
    quants_results[data_file] = result

### Bias correction method check (1) ###

##### Verify bias-corrected historical CMIP6 is consistent with ERA-5, while non-bias-corrected isn't. Correction should reduce the bias (so quantiles diffs should be closer to zero)

In [67]:
for c in cells:
    print(f'cell : {c}')
    print('non corrected')
    print(quants_results[data_dict['coarse']['ERA-5']][c].values-quants_results[data_dict['coarse']['cmip6']['historical']][c].values)
    print('corrected')
    print(quants_results[data_dict['coarse']['ERA-5']][c].values-quants_results[data_dict['coarse']['bias_corrected']['historical']][c].values)

cell : (41.5, 1.5)
non corrected
[2.56773102 2.15558777 1.97580566 2.34085083 3.32855225 3.06008911
 2.49491577 2.06820374 1.76759369]
corrected
[1.12341064 0.98111115 0.84924927 0.58791351 0.47463989 0.6499939
 0.42756042 0.24011841 0.12395844]
cell : (39.5, -83.5)
non corrected
[0.24749176 0.93431091 2.05977478 4.80963898 4.96957397 3.92437744
 3.11152039 2.97287292 3.2295816 ]
corrected
[-0.3089743  -0.37416229 -0.40345154  0.03394318  0.18711853  0.39482117
  0.30922241  0.26907196  0.19752075]
cell : (-5.5, -49.5)
non corrected
[ 2.81440765  2.70875092  2.26695557  2.23838806  2.25622559 -0.85774231
 -2.69372253 -3.00125885 -3.31266205]
corrected
[0.79361389 0.74092407 0.57475586 0.46287537 0.47612    0.40182495
 0.38529968 0.40956726 0.41493103]


### Bias correction method check (2) ###

##### Verify CMIP6 absolute changes in quantiles across time are preserved after QDM bias correction.

In [14]:
for c in cells:
    print(f'cell : {c}')
    print('non corrected')
    print(quants_results[data_dict['coarse']['cmip6']['ssp370']][c].values-quants_results[data_dict['coarse']['cmip6']['historical']][c].values)
    print('corrected')
    print(quants_results[data_dict['coarse']['bias_corrected']['ssp370']][c].values-quants_results[data_dict['coarse']['bias_corrected']['historical']][c].values)

cell : (41.5, 1.5)
non corrected
[3.68578522 3.20266876 3.14243774 3.14105988 3.70657349 4.42950439
 4.95620422 4.98266907 4.68912598]
corrected
[3.87143188 3.08726807 3.14093018 3.51397705 3.96615601 4.8757019
 5.29766846 5.19889526 4.94866333]
cell : (39.5, -83.5)
non corrected
[4.91748657 2.47857208 2.54364014 3.39523315 3.83833313 4.31138611
 4.07737732 3.99814301 3.65487091]
corrected
[5.15460327 2.65206299 2.66172485 3.64028931 4.08895874 4.36862183
 4.30161133 4.03612671 3.95448853]
cell : (-5.5, -49.5)
non corrected
[2.24838043 2.55540619 2.65269165 3.25076294 3.73539734 4.23408508
 3.90587769 3.66032257 3.33886932]
corrected
[2.55775635 2.8618042  3.00776978 3.49560547 4.03863525 4.00012207
 3.64265137 3.59101563 3.56733765]
